# Imports

* Instead of original dataset provided, we work with combination of (2019+2015) dataset
* Instead of .png , dataset uses .jpeg pictures
* The dataset creators also ensures that they have eliminated the ultra-dark images
* COMPETITION DATASET : 5590 images (aptos2019-blindness-detection)
* CONTRIBUTED DATASET : ~ 35k images (diabetic-retinopathy-resized)
* special thanks to : @ILOVESCIENCE for this dataset

https://www.kaggle.com/datasets/tanlikesmath/diabetic-retinopathy-resized?select=trainLabels_cropped.csv

NOTE : **Quadratic Kappa Metric** is the same as **cohen kappa metric** in Sci-kit learn @ sklearn.metrics.cohen_kappa_score when **weights = 'Quadratic'**. 

In [ ]:
import numpy as np 
import pandas as pd 
import os
from tqdm import tqdm 
import warnings
warnings.filterwarnings('ignore')
import time

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

# sklearn
from sklearn.utils import class_weight, shuffle
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix, precision_score, recall_score
import sklearn.utils.class_weight as class_weight

# torch
import torchvision
from torchvision import transforms
import torch
import torch.nn as nn
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchmetrics

# lightening
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint, RichProgressBar, RichModelSummary

# Analysing Data

In [ ]:
train_data = pd.read_csv('/kaggle/input/diabetic-retinopathy-resized/trainLabels_cropped.csv')
test_data = pd.read_csv('/kaggle/input/aptos2019-blindness-detection/test.csv')

train_data.rename(columns={'level': 'labels'}, inplace=True)
train_data = train_data.drop(["Unnamed: 0.1", "Unnamed: 0"], axis = 1)

class_labels = train_data['labels'].unique() 
print("Target classes: {}".format(class_labels))

train_data['eye_type'] = train_data['image'].map(lambda x: x.split('_')[1])
train_data.head(2)

In [ ]:
class_counts = train_data['labels'].value_counts()
class_labels = class_counts.index.tolist()
class_sizes = class_counts.tolist()

class_explode = [0.1] * len(class_labels)  

fig, axs = plt.subplots(1, 2, figsize=(14, 7))

# Subplot 1: Distribution of Training and Testing Data
axs[0].pie([len(train_data), len(test_data)], explode=[0, 0.1], labels=['Train Data', 'Test Data'], shadow=True, autopct='%1.1f%%', startangle=100)
axs[0].set_title('Distribution of Test and Training Data')

# Subplot 2: Distribution of Classes in Training Data
axs[1].pie(class_sizes, explode=class_explode, labels=class_labels, shadow=True, autopct='%1.1f%%', startangle=45)
axs[1].set_title('Distribution of Classes in Training Data')

plt.show()

*As we can see data is highly imbalanced and we have approx 10 times more images for class 0 than other classes*

* We must use weighted loss, to weigh the loss of small classes more than the dominant one

In [ ]:
tr_path = '/kaggle/input/diabetic-retinopathy-resized/resized_train_cropped/resized_train_cropped'
tst_path = '/kaggle/input/aptos2019-blindness-detection/test_images'

train_data['path'] = train_data['image'].map(lambda x: os.path.join(tr_path,'{}.jpeg'.format(x)))
test_data['path'] = test_data['id_code'].map(lambda x: os.path.join(tst_path,'{}.png'.format(x)))

print(train_data['eye_type'].value_counts())
train_data = train_data.iloc[:len(train_data) // 2]
print(train_data.shape)
train_data.head(2)

In [ ]:
train_data.to_csv('tr_df.csv', index=  False)
test_data.to_csv('tst_df.csv', index=  False)

# Image Plotting 

In [ ]:
N = 12
random_idxs = np.random.choice(train_data.index , N)

fig, axs = plt.subplots(4,3,figsize = (15,20))
for i in range(N):
    r,c = i//3 , i%3
    idx = random_idxs[i]
    img_path, eye_type = train_data.iloc[idx, 3],  train_data.iloc[idx, 2]
    img = cv2.imread(img_path)
    axs[r,c].imshow(img)
    axs[r,c].set_title('{a}_{b}'.format(a=img.shape , b = eye_type))
    axs[r,c].set_xticks([])
    axs[r,c].set_yticks([])
    
plt.tight_layout()

# Transformations & Splitting

The images are of different shapes, they can't be batched directly. Hence we can apply the following transforms :

* resize_with_pad 
* Centre_crop
* We can't apply vertical_flip , as it will not match reality
* We need not to apply horizontal_flip, as there are enough no. of (left_eye ,right_eye) images
* We can apply affine transformations for model robustness.

We can skip the first 2 transformations as every pre-trained model do these transformations internally.


In [ ]:
img_dim = 600
img_transforms = transforms.Compose([
        torchvision.transforms.Resize(size=(img_dim,img_dim)), 
        torchvision.transforms.GaussianBlur(kernel_size = 5, sigma = (20,20)),
        transforms.RandomAffine(degrees=5, translate=(0.1, 0.1), scale=(0.8, 1.2)),
        transforms.ToTensor(),
   ])

# Creating train and validation sets

X, y = train_data.path, train_data.labels
X_tr, X_val, Y_tr,Y_val = train_test_split(X, y, test_size=0.2,
                                                      stratify=y, random_state=42)
X_tr.reset_index(drop = True, inplace = True)
Y_tr.reset_index(drop = True, inplace = True)
X_val.reset_index(drop = True, inplace = True)
Y_val.reset_index(drop = True, inplace = True)
print(X_tr.shape, Y_tr.shape, X_val.shape, Y_val.shape)

# Image Preprocessing

In [ ]:
# # Plotting pixel_histogram of pixels of each channel

# img = cv2.imread(X_tr.iloc[0])
# img_channel_list = [img , img[:,:,0] , img[:,:,1] ,img[:,:,2]]
# fig, axs = plt.subplots(2,2 , figsize= (15,8))

# axs[0,0].imshow(img)
# axs[0,0].set_xticks([])
# axs[0,0].set_yticks([])
# labels = [i for i in range (256)]
# for i in range(1,4):
#     r,c  = i//2,i%2
#     axs[r,c].hist(img_channel_list[i])
    
#     axs[r,c].set_xticks(labels, labels, rotation=45)
# #     axs[r,c].set_xticklabels([i for i in range (256)], rotation=45)
#     axs[r,c].set_yticks([])
#     axs[r,c].set_title('Channel no. {x}'.format(x=i-1))

Here, each channel has clear distinction between low pixel values and rest pixel values (0_th and 1_st towers in each channel)

So, even if we merge these channels, rest small towers may be compensated by other channels, but the distinction between 0_th and 1_st channel stays

Hence, we can use gray_scale image for getting img_contours for cropping

In [ ]:
class PreProcessing:
    def __init__(self, no_channels, tol=7, sigmaX=30):
        self.no_channels = no_channels
        self.tol = tol
        self.sigmaX = sigmaX

    def cropping_2D(self, img, is_cropping = False):    # for Cropping the extra dark part of the GRAY images

        mask = img>self.tol      #  thresholding the img at a value of self.tol
        '''
         --> If the mask image has any pixels set to 1 in a row, then the corresponding row in img will be kept. 
         --> If the mask image has any pixels set to 1 in a column, then the corresponding column in img will be kept. 
         --> Otherwise, the row or column in the input image will be removed.
        '''
        return img[np.ix_(mask.any(1),mask.any(0))]

    def cropping_3D(self, img, is_cropping = False):  # for Cropping the extra dark part of the RGB images

        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>self.tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # if image is too dark we return the image
            return img 
        else:
            img1 = img[:,:,0][np.ix_(mask.any(1),mask.any(0))]  #for channel_1 (R)
            img2 = img[:,:,1][np.ix_(mask.any(1),mask.any(0))]  #for channel_2 (G)
            img3 = img[:,:,2][np.ix_(mask.any(1),mask.any(0))]  #for channel_3 (B)         
            img = np.stack([img1,img2,img3],axis=-1)
        return img

    def Gaussian_blur(self, img, is_gaussianblur = False):
        # adding Gaussian blur (image smoothing technique) thus reducing noise in image.

        return cv2.addWeighted ( img,4, cv2.GaussianBlur( img , (0,0) , self.sigmaX) ,-4 ,128)

    def draw_circle(self,img, is_drawcircle = True):
        #This function is used for drawing a circle from the center of the image.

        x = int(self.img_width/2)
        y = int(self.img_height/2)
        r = np.amin((x,y))     # finding radius to draw a circle from the center of the image
        circle_img = np.zeros((img_height, img_width), np.uint8)
        cv2.circle(circle_img, (x,y), int(r), 1, thickness=-1)
        img = cv2.bitwise_and(img, img, mask=circle_img)
        return img

    def image_preprocessing(self, img, is_cropping = True, is_gaussianblur = True):
#         if img.shape[2] == 2:
#             img = self.cropping_2D(img, is_cropping)  #calling cropping_2D for a GRAY image
#         else:
        img = self.cropping_3D(img, is_cropping)  #calling cropping_3D for a RGB image
#         img = cv2.resize(img, (self.img_height, self.img_width))  # resizing the image with specified values
#         img = self.draw_circle(img)  #calling draw_circle
        img = self.Gaussian_blur(img, is_gaussianblur) #calling Gaussian_blur
        return img

In [ ]:
# channels = 3
# split_size = 0.2
# class_labels = {0: 'No DR[0]',1: 'Mild[1]', 2: 'Moderate[2]', 3: 'Severe[3]', 4: 'Proliferative DR[4]'}
# obj = PreProcessing(channels, sigmaX = 45)
# random_img_paths = np.random.choice(X_tr, 3)

# fig, axs = plt.subplots(3,2, figsize = (15,15))

# for i in range(3):
#     raw_img = cv2.imread(random_img_paths[i])
#     axs[i,0].imshow(raw_img)
#     axs[i,0].set_title('unprocessed : {x}'.format(x = raw_img.shape))
#     axs[i,0].set_xticks([])
#     axs[i,0].set_yticks([])
    
#     processed_img = obj.image_preprocessing(raw_img)
#     axs[i,1].imshow(processed_img)
#     axs[i,1].set_title('processed :{x}'.format(x = processed_img.shape))
#     axs[i,1].set_xticks([])
#     axs[i,1].set_yticks([])
    
# plt.tight_layout()

# Input PipeLine

In [ ]:
class_weights = class_weight.compute_class_weight(class_weight= 'balanced',classes=  np.unique(Y_tr), y= Y_tr)
class_weights = torch.tensor(class_weights,dtype=torch.float)

class Classifier(pl.LightningModule):
    def __init__(self, model_obj):
        super().__init__()
        self.model = model_obj.model
        self.config = model_obj.config
        self.layer_lr = model_obj.layer_lr
        
        self.kappa = torchmetrics.classification.MulticlassCohenKappa(num_classes = self.config['num_classes'], weights = 'quadratic')
        self.accuracy = torchmetrics.Accuracy(task = 'multiclass' , num_classes = self.config['num_classes'])
        self.criterion = torch.nn.CrossEntropyLoss(weight = class_weights)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = self.criterion(y_hat, y.long())
        self.accuracy(y_hat, y)
        self.kappa(y_hat, y)
        self.log("train_acc", self.accuracy, on_epoch=True,prog_bar=True, logger=True)
        self.log("train_loss", loss, on_epoch=True, prog_bar=True, logger=True)
        self.log("train_kappa", self.kappa, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = self.criterion(y_hat, y.long())
        self.accuracy(y_hat, y)
        self.kappa(y_hat, y)
        self.log("val_acc", self.accuracy, on_epoch=True,prog_bar=True, logger=True)
        self.log("val_loss", loss, on_epoch=True, prog_bar=True, logger=True)
        self.log("val_kappa", self.kappa, on_epoch=True, prog_bar=True, logger=True)
        return loss
  
    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = self.criterion(y_hat, y)
        self.accuracy(y_hat, y)
        self.kappa(y_hat, y)
        self.log("test_acc", self.accuracy, on_epoch=True,prog_bar=True, logger=True)
        self.log("test_loss", loss, on_epoch=True, prog_bar=True, logger=True)
        self.log("test_kappa", self.kappa, on_epoch=True, prog_bar=True, logger=True)
        return loss
  
    def configure_optimizers(self):
        optim =  torch.optim.Adam(self.layer_lr, lr = self.config['lr'])   # https://pytorch.org/docs/stable/optim.html
        lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=3, factor=0.5, threshold=0.001, cooldown =2,verbose=True)
        return [optim], [{'scheduler': lr_scheduler, 'interval': 'epoch', 'monitor': 'train_loss', 'name': 'lr_scheduler'}]

#___________________________________________________________________________________________________________________
class MyDataset(Dataset):
    
    # defining values in the constructor
    def __init__(self , X,y,img_transforms, img_processing_obj, apply_processing = True):
        self.X = X
        self.Y = torch.tensor( y.values, dtype=torch.float32)
        self.total = len(self.X)
        self.img_transforms = img_transforms
        self.img_processing_obj = img_processing_obj
        self.apply_processing = apply_processing
    
    # Getting the data samples
    def __getitem__(self, idx):
        y =  self.Y[idx]
        img_path = self.X.iloc[idx]
        img_tensor = Image.open(img_path)   # np.asarray()

#         if self.apply_processing:
#             img_tensor = self.img_processing_obj.image_preprocessing(img_tensor)

        img_tensor = self.img_transforms(img_tensor)
        return img_tensor, y
  
    def __len__(self):
        return self.total
  
            


# DataLoaders

In [ ]:
obj = PreProcessing(3, sigmaX = 45)

tr_dataset = MyDataset(X_tr, Y_tr, img_transforms, obj)
val_dataset = MyDataset(X_val, Y_val, img_transforms , obj)

num_workers = 4
BATCH_SIZE = 32
tr_loader = DataLoader(tr_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers)

# Callbacks

In [ ]:
early_stop_callback = EarlyStopping(
   monitor='val_loss',
   min_delta=0.00,
   patience=5,
   verbose=True,
   mode='min'
)

checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    save_top_k=2,
    verbose=True,
 )
rich_progress_bar = RichProgressBar()

rich_model_summary = RichModelSummary(max_depth=2)

# Transfer Learning

Now is te time to do transfer_learning, here is my plan for code distribution

* I create seperate object for each pre-trained model customised for the setting of that model
* I do transfer-learning on each object, then, do analysis of models against the class they are weak/strong at configuring
* Then do ensembling, to further enhance the results

**Models to be analysed : ResNeXt-50 , XceptionNet, ViT**

In [ ]:
class ResNeXt():
  # https://pytorch.org/vision/main/models/generated/torchvision.models.resnet101.html
    def __init__(self, config ):
        self.config = config
        self.resnext =  torchvision.models.resnet101(weights = 'ResNet101_Weights.DEFAULT', progress = True)
        self.base_model = nn.Sequential(*list(self.resnext.children())[:-1])

        self.__create_model__()
        self.layer_lr = [{'params' : self.base_model.parameters()},{'params': self.head.parameters(), 'lr': self.config['lr'] * 100}]

    def __create_model__(self):
        self.head = nn.Sequential(
                    Dense(0.2 , 2048 ,1024), 
                    Dense(0.15, 1024, 512) ,
                    Dense(0, 512, self.config['num_classes'])
        )
        self.model = nn.Sequential(
                    self.base_model ,
                    nn.Flatten(), 
                    self.head ,
                        )
    def forward(self, x):
        return self.model(x)
#_______________________________________________________________________________________________________________
class EffecientNet():
  # https://pytorch.org/vision/stable/models/generated/torchvision.models.efficientnet_v2_l.html#torchvision.models.efficientnet_v2_l
    def __init__(self, config):
        self.config = config
        self.enet = torchvision.models.efficientnet_v2_l( weights='DEFAULT' , progress = True)    # 'DEFAULT'  : 'IMAGENET1K_V1'
        self.base_model = nn.Sequential(*list(self.enet.children())[:-1])

        self.__create_model__()
        self.layer_lr = [{'params' : self.base_model.parameters()},{'params': self.head.parameters(), 'lr': self.config['lr'] * 100}]

    def __create_model__(self): 
        self.head = nn.Sequential(
                    Dense(0.4 , 1280 ,512), 
                    Dense(0, 512, self.config['num_classes'])
        )
        self.model = nn.Sequential(
                      self.base_model ,
                      self.head
                            )      
    def forward(self, x):
        x =  self.model(x) 
        return x

#__________________________________________________________________________________________________________________

class ViT():
  # https://pytorch.org/vision/stable/models/generated/torchvision.models.efficientnet_v2_l.html#torchvision.models.efficientnet_v2_l
    def __init__(self, config):
        self.config = config
        self.vit = torchvision.models.vit_b_32( weights='DEFAULT' , progress = True)    # 'DEFAULT'  : 'IMAGENET1K_V1'
        self.base_model = nn.Sequential(*list(self.vit.children())[:-1])

        self.__create_model__()
        self.layer_lr = [{'params' : self.base_model.parameters()},{'params': self.head.parameters(), 'lr': self.config['lr'] * 100}]

    def __create_model__(self): 
        self.head = nn.Sequential(
                    Dense(0.2 , 768 ,512), 
                    Dense(0, 512, self.config['num_classes'])
        )
        self.model = nn.Sequential(
                      self.base_model ,
                      self.head
                            )      
    def forward(self, x):
        x =  self.model(x) 
        return x
#___________________________________________________________________________________________________________________

class Dense(nn.Module):
    def __init__(self, drop ,in_size, out_size):
        super(Dense ,self).__init__()
        self.dropout = nn.Dropout(drop)
        self.linear = nn.Linear(in_size, out_size)
        self.prelu = nn.PReLU()

    def forward(self, x):
        x = self.dropout(x)
        x = self.linear(x)
        x = self.prelu(x)
        return x

# ResNeXt-50

In [ ]:
resNeXt_dir = os.path.join('/kaggle/working/' , 'resNeXt_50')
if not os.path.isdir(resNeXt_dir):
    os.mkdir(resNeXt_dir)
    
config = {
    'dir' : resNeXt_dir, 

    'num_classes' : 5,

    'BATCH_SIZE' : 32,
    'lr' : 0.00001,

    'ckpt_file_name' : '{epoch}-{val_loss:.2f}-{val_acc:.2f}-{val_kappa:.2f}',
    'model_name' : 'resNeXt_50',
}

In [ ]:
model_obj = ResNeXt(config)
model = Classifier(model_obj)

checkpoint_callback.dirpath = os.path.join(config['dir'], 'ckpts')
checkpoint_callback.filename = config['ckpt_file_name']

logger = TensorBoardLogger(resNeXt_dir+"/tb_logs")

trainer = Trainer(callbacks=[early_stop_callback, checkpoint_callback, rich_progress_bar, rich_model_summary], 
                  accelerator = 'gpu' ,max_epochs=1, logger=[logger])  
 
trainer.fit(model, tr_loader, val_loader)

# EfficientNet_v2

In [ ]:
efficientnet_v2_dir = os.path.join('/kaggle/working/' , 'efficientnet_v2')
if not os.path.isdir(efficientnet_v2_dir):
    os.mkdir(efficientnet_v2_dir)
    
config = {
    'dir' : efficientnet_v2_dir, 

    'num_classes' : 5,

    'BATCH_SIZE' : 32,
    'lr' : 0.00001,

    'ckpt_file_name' : '{epoch}-{val_loss:.2f}-{val_acc:.2f}-{val_kappa:.2f}',
    'model_name' : 'efficientnet_v2',
}

In [ ]:
model_obj = EffecientNet(config)
model = Classifier(model_obj)

checkpoint_callback.dirpath = os.path.join(config['dir'], 'ckpts')
checkpoint_callback.filename = config['ckpt_file_name']

logger = TensorBoardLogger(efficientnet_v2_dir+"/tb_logs")

trainer = Trainer(callbacks=[early_stop_callback, checkpoint_callback, rich_progress_bar, rich_model_summary], 
                  accelerator = 'gpu' ,max_epochs=1, logger=[logger])  
 
trainer.fit(model, tr_loader, val_loader)

# ViT-b32

In [ ]:
vit_b_32_dir = os.path.join('/kaggle/working/' , 'efficientnet_v2')
if not os.path.isdir(vit_b_32_dir):
    os.mkdir(vit_b_32_dir)
    
config = {
    'dir' : vit_b_32_dir, 

    'num_classes' : 5, 

    'BATCH_SIZE' : 32, 
    'lr' : 0.00001, 

    'ckpt_file_name' : '{epoch}-{val_loss:.2f}-{val_acc:.2f}-{val_kappa:.2f}' ,
    'model_name' : 'vit_b_32', 
}

In [ ]:
model_obj = ViT(config)
model = Classifier(model_obj)

checkpoint_callback.dirpath = os.path.join(config['dir'], 'ckpts')
checkpoint_callback.filename = config['ckpt_file_name']

logger = TensorBoardLogger(vit_b_32_dir+"/tb_logs")

trainer = Trainer(callbacks=[early_stop_callback, checkpoint_callback, rich_progress_bar, rich_model_summary], 
                  accelerator = 'cpu' ,max_epochs=1, logger=[logger])  
 
trainer.fit(model, tr_loader, val_loader)